In [1]:
import pandas as pd
from collections import defaultdict
import random
import pickle
from datetime import datetime, timedelta

import json
import os
import sys
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_auc_score, roc_curve,
                             auc, log_loss, f1_score, accuracy_score,
                             recall_score, precision_score, classification_report)
import warnings
import argparse



In [2]:
df = pd.read_pickle('0301_data_unencode.pkl')


In [3]:
# # print(df.columns)
cols = [col for col in df.columns if col not in ['category', 'feature','scenario','audience','brand','b_c','c_s','a_c','c_f','b_c_s','a_b_c','b_c_f','a_c_s','c_f_s','a_c_f','a_c_b_s','b_c_f_s','a_c_f_s','a_b_c_f_s','label']] +['category', 'feature','scenario','audience','brand','b_c','c_s','a_c','c_f','b_c_s','a_b_c','b_c_f','a_c_s','c_f_s','a_c_f','a_c_b_s','b_c_f_s','a_c_f_s','a_b_c_f_s','label']
df = df[cols]

In [4]:
import pandas as pd

# 假设 df 是你的 DataFrame
columns_to_reindex = ['category', 'feature', 'scenario', 'audience', 'brand', 'b_c', 'c_s', 
                      'a_c', 'c_f', 'b_c_s', 'a_b_c', 'b_c_f', 'a_c_s', 'c_f_s', 
                      'a_c_f', 'a_c_b_s', 'b_c_f_s', 'a_c_f_s', 'a_b_c_f_s']

# 对每列重新编码
for col in columns_to_reindex:
    df[col], unique_values = pd.factorize(df[col])
    print(f"Column {col} has {len(unique_values)} unique values: {unique_values}")


Column category has 12390 unique values: Index(['孕妇奶粉', '乐福鞋', '玩偶', '卸妆液', '鞋子', '半身裙', '七巧板', '餐椅', '防晒衣', '打底衫',
       ...
       '口袋霜', '儿童马桶圈', '牛轧饼', '耳机清洁笔刷', '汉服连衣裙', '洁颜膏', '卷发夹板', '背篓', '线管',
       '营养米糊'],
      dtype='object', length=12390)
Column feature has 307493 unique values: Index(['专用;舒孕', '舒孕', '专用', '轻盈', '', '眼唇可用', '英伦;麦穗', '2025春季;玛丽珍;麦穗',
       '玛丽珍;羊皮', '2025春季;玛丽珍;羊皮;麦穗',
       ...
       '密集滋养;轻盈', '可调节;洋气;薄款', '可调节;薄款', '铁锌钙', '20g;试用装', '5包装;木浆;画画',
       '5包装;画画', '木浆;画画', '5包装;木浆', '不透人;梦幻;透光'],
      dtype='object', length=307493)
Column scenario has 2585 unique values: Index(['', '日常穿着', '学习', '家用', '运动', '日常佩戴', '伤口护理', '商场;通勤', '通勤', '商场',
       ...
       '打豆浆', '打豆浆;煲汤', '发热;感冒', '发热', '砍柴', '买菜;收纳;采摘', '采摘', '买菜;收纳',
       '收纳;采摘', '买菜;采摘'],
      dtype='object', length=2585)
Column audience has 2354 unique values: Index(['', '孕妇', '男士', '女朋友;女生;朋友', '敏感肌', '女士', '男女儿童', '小学生;男女儿童', '小学生',
       '婴儿',
       ...
       '农民;消费者', '新手;男女;青年大

In [5]:
train_test_size=0.2
user = df["oaid"].unique().tolist()
user_train, user_test = train_test_split(user, test_size=train_test_size, random_state=42)
X_train, X_test= df[df["oaid"].isin(user_train)], df[df["oaid"].isin(user_test)]


X_train.drop(columns=['ds', 'oaid', 'ici','day','title','max_similarity'],inplace = True,axis=1)
X_test.drop(columns=['ds', 'oaid', 'ici','day','title','max_similarity'],inplace = True,axis=1)

/tmp/ipykernel_2225102/1320648794.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(columns=['ds', 'oaid', 'ici','day','title','max_similarity'],inplace = True,axis=1)
/tmp/ipykernel_2225102/1320648794.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(columns=['ds', 'oaid', 'ici','day','title','max_similarity'],inplace = True,axis=1)


In [6]:
X_train.to_pickle('data/train_data.pkl')
X_test.to_pickle('data/test_data.pkl')